# Partial Scanner
This can Scan computer pictures from a good resolution camera ignoring the other details present around the screen. It can also scan a page provided the angle and resolution of camera is helpful for proper Image assessment. I have also attached an output.


Furthermore, this can also be used on a live WebCam and Camera but the resolution has to be good otherwise it won't fetch the details. As I didn't have a good quality camera, I went ahead with normal picture uploading and checking.


I learned OpenCV from several YouTube tutorials and many of my concepts have been taken from them. I have also made several codes for different tasks and they are uploaded on my GitHub for future use.  

In [39]:
import cv2
import numpy as np

#The width and height parameters are used for resizing the image. This is stored in dimensions to be used later on

widthImg=1500
heightImg =1200
dims = (widthImg,heightImg)

# Processing the Image to include Blur and Canny Edge for Edge Detection

1.) imgGray is used to convert image into GrayScale as Blur and Canny Edges are applied on GrayScale.

2.) GaussianBlur is used to include Noise inside the image.

3.) Canny Edge Detection is used for finding edges of objects in images.

4.) Kernel is used for Dilation and to erode the lines inside the Images. This makes the lines inside images a   bit more wide.

In [40]:
def Process(img):
    Gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    Blur = cv2.GaussianBlur(Gray,(5,5),1)
    Canny = cv2.Canny(Blur,200,200)
    kernel = np.ones((5,5))
    Dial = cv2.dilate(Canny,kernel,iterations=2)
    Thres = cv2.erode(Dial,kernel,iterations=1)
    return Thres

# Getting contours around objects in images
1.) "Biggest" variable here stores the Numerical array values for objects. This means that if any object detected is larger than previous value, it will replace the previous value.

2.) "contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)" ==> This is used to find all the contours present inside the image and store it in contours.

3.) A loop is passed so that all contours are iterated.

4.) Area variable is used to store contourAreas that are being iterated over in the loop. This means, new values will be successively passed into the loop.

5.) Minimum threshold for area is kept and if the area of a contour is more than the threshold, it is stored inside perimeter.

6.) Approx variable is used such that points are generated which is to be used to find the feature points of contours. 

7.) At the end, Contours are drawn around the feature points and the copy of the original image.

In [41]:
def Contour(img):
    biggest = np.array([])
    mArea = 0
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>50:
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            if area > mArea and len(approx) == 4:
                biggest = approx
                mArea = area
    cv2.drawContours(imgContour, biggest, -1, (255, 0, 0), 20)
    return biggest

#  Reordering of feature points of the Biggest Contour found in getContours() function
1.) The reshape function is used to split the feature points into 4 ROWS and 2 COLUMNS such that 4 points are found.

2.) np.zeros makes a new array of 4 ROWS and 2 COLUMNS with data type of 32 bit integer.

3.) myPoints.sum(1) is used to adding all the numerical values of features points into a single value accross a row. This is done because of following reason ==> The highest value for a particular row suggests that the point is end point for contour. Similarly, the lowest value represents the starting point. The 3rd and 4th point can be found by addition and subtraction.

4.)np.argmin(add) returns the Indices of lowest value inside the new Array which means Starting Point.
Similarly, the highest value is returned for the end point which is the third Index.

5.)Similarly for the left and right coordinates, the minimum and maximum difference is taken from subtraction of array.

In [42]:
def reorder (myPt):
    myPt = myPt.reshape((4,2))
    myPtNew = np.zeros((4,1,2),np.int32)
    add = myPt.sum(1)
    myPtNew[0] = myPt[np.argmin(add)]
    myPtNew[3] = myPt[np.argmax(add)]
    diff = np.diff(myPt,axis=1)
    myPtNew[1] = myPt[np.argmin(diff)]
    myPtNew[2] = myPt[np.argmax(diff)]
    return myPtNew

# Warping of images 
The point 1 and point 2 are used for transformation. The point 2 consists of coordinates of the original image and the point 1 consits of the reordered feature points of the biggest contour

2.) matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgOutput = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
    This is used to create the matrix and then finally the image is wrapped into using the original frame and the 
    matrix just created.
    
3.) At the end, the image is resized again.

In [43]:
def Warp(img,biggest):
    biggest = reorder(biggest)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [widthImg, 0], [0, heightImg], [widthImg, heightImg]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgOutput = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
 
    imgCropped = imgOutput[20:imgOutput.shape[0]-20,20:imgOutput.shape[1]-20]
    imgCropped = cv2.resize(imgCropped,(widthImg,heightImg))
 
    return imgCropped

# Stacking Images in Horizontal and Vertical Stacks
This is something I used from the a blog post to make my task easier as it is cumbersome to merge grayScale image with a normal image.

In [44]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

# Calling functions and reading the images and outputting them

In [48]:
img1 = cv2.imread('note5.jpg') 
img = cv2.resize(img1,(widthImg,heightImg))
imgContour = img.copy()
 
imgThres = preProcessing(img)
biggest = getContours(imgThres)
if biggest.size !=0:
    imgWarped=getWarp(img,biggest)
    imageArray = ([imgContour, imgWarped, imgThres])
else:
    imageArray = ([imgContour, img,imgThres])
 
stackedImages = stackImages(2,imageArray,)
width = 1080
height = 540
final = cv2.resize(stackedImages,(width,height))
cv2.imshow("WorkFlow", final)
cv2.waitKey(0)

-1